# Analysis of the (danger) High Voltage sweeps
And compared to the fixed wall potential and piecewise sims.

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.utils as spu

In [2]:
importlib.reload(lpu)
importlib.reload(spl)

<module 'flopter.spice.splopter' from '/home/jleland/coding/projects/flopter/flopter/spice/splopter.py'>

# Preamble of necessary stuff

This bit creates all teh necessary variables for plotting to commence, including loading the dataset, creating a denormaliser, extracting relevant probe dimensions from an input file and defining a few functions for ease of manipulation. 

In [3]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [4]:
lowsweep_ds = xr.load_dataset('new_sheath_exp_ivs.nc')
piecewise_ds = xr.load_dataset('piecewise_sheath_exp_ivs.nc')

In [5]:
skippable_scans = set(['marconi/spice2/piecewise/flat_flush_piecewise']) 
single_sims = set()

In [29]:
# scans_searchstr = '*/*/sheath_exp/*'
# scans_searchstr = '*/*/new_sheath_exp/*'
# scans_searchstr = '*/*/piecewise/*'
scans_searchstr = '*/*/sheath_exp_f*p/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)



scans = list(scans)
scans.sort()

In [7]:
for i, scan in enumerate(scans):
    print(f"[{i}]: {scan}")
    for j, run in enumerate(all_run_dirs[scan]):
        print(f"\t[{i},{j}]: {'/'.join(run.split('/')[-2:])}")

[0]: marconi/spice2/sheath_exp_flp/flat_10_highV
	[0,0]: flat_10_highV/alpha_yz_-10.0
[1]: marconi/spice2/sheath_exp_flp/flat_8-12_highV
	[1,0]: flat_8-12_highV/alpha_yz_-12.0
	[1,1]: flat_8-12_highV/alpha_yz_-8.0
[2]: marconi/spice2/sheath_exp_fwp/angled_flush_as
	[2,0]: angled_flush_as/alpha_yz_-10.0
	[2,1]: angled_flush_as/alpha_yz_-12.0
	[2,2]: angled_flush_as/alpha_yz_-2.0
	[2,3]: angled_flush_as/alpha_yz_-20.0
	[2,4]: angled_flush_as/alpha_yz_-3.0
	[2,5]: angled_flush_as/alpha_yz_-30.0
	[2,6]: angled_flush_as/alpha_yz_-4.0
	[2,7]: angled_flush_as/alpha_yz_-5.0
	[2,8]: angled_flush_as/alpha_yz_-6.0
	[2,9]: angled_flush_as/alpha_yz_-8.0
[3]: marconi/spice2/sheath_exp_fwp/angled_recessed_as
	[3,0]: angled_recessed_as/alpha_yz_-10.0
	[3,1]: angled_recessed_as/alpha_yz_-12.0
	[3,2]: angled_recessed_as/alpha_yz_-2.0
	[3,3]: angled_recessed_as/alpha_yz_-20.0
	[3,4]: angled_recessed_as/alpha_yz_-3.0
	[3,5]: angled_recessed_as/alpha_yz_-30.0
	[3,6]: angled_recessed_as/alpha_yz_-4.0
	[3,7]

In [12]:
spl_path = spice_dir / all_run_dirs[scans[7]][0]
print(spl_path)

/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_flush_as/alpha_yz_-10.0


## Start with the one

In [13]:
bups = list(spl_path.glob('backup*'))
bups.sort()
final_state_path = spl_path / bups[-1].name

In [21]:
splopter =  spl.Splopter(final_state_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True, check_voltage_error_fl=False)

splopter.prepare(find_se_temp_fl=False, homogenise_fl=True, backup_concat_fl=False)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_flush_as/alpha_yz_-10.0/backup_20201126-1028) doesn't seem to be valid.
Continuing anyway.


In [30]:
fig, ax = plt.subplots()

ax.plot(splopter.tdata.t_dict['ProbePot'])

for i in range(4):
    line_pos = i * len(splopter.tdata.t_dict['ProbePot']) / 4
    ax.axvline(x=line_pos, **c.AX_LINE_DEFAULTS)

In [16]:
t, V, I_e, I_i = splopter.get_tdata_raw_iv()
I_tot = (I_e + I_i)

fig, ax = plt.subplots(2, sharex=True)

ax[0].plot(I_tot[::100])
ax[1].plot(I_e[::100], label='e')
ax[1].plot(I_i[::100], label='i')
ax[1].legend()

In [23]:
swp_len = splopter.parser.get_sweep_length(len(I_tot), V)

fig, ax = plt.subplots()
# splopter.raw_data.plot(ax=ax)
raw_len = len(splopter.raw_data['V'])
print(swp_len, raw_len)

ax.plot(splopter.raw_data['V'][::100])
ax.plot(np.arange(swp_len/100, raw_len/100), splopter.raw_data['V'][swp_len::100])


457978 915955


In [24]:
swp_len = int(raw_len / 2)

fig, ax = plt.subplots(2)
# ax.plot(splopter.raw_data['V'][::100], splopter.raw_data['I'][::100])
ax[0].plot(splopter.raw_data['V'][::100])
ax[0].plot(np.arange(swp_len/100, raw_len/100)+1, splopter.raw_data['V'][swp_len::100])
ax[1].plot(splopter.raw_data['I'][::100])

In [26]:
losw_plot_ds = lowsweep_ds.sel(theta_p=0.0, recession=0.0, theta=10.0)
pw_plot_ds = piecewise_ds.sel(theta=10.0)

fig, ax = plt.subplots()
splopter.iv_data.plot(ax=ax, label='high-V sweep')
(-pw_plot_ds['current']).plot.line(x='voltage', hue='theta', ax=ax, label='piecewise')
losw_plot_ds['current'].plot.line(x='voltage', ax=ax, label='low-V sweep')
ax.axhline(y=0, **c.AX_LINE_DEFAULTS)
ax.legend()

Noticed a mistake in the input file, the probes are not at the correct starting voltage - should be at -20 but are at -10 so there's a jump of voltage at the start of the sweep. Will probably need to redo.

## Compare the floating and fixed wall potential cases


In [27]:
fwp_dir_path = spice_dir / all_run_dirs[scans[5]][0]
print(fwp_dir_path)
fwp_backup_path = list(fwp_dir_path.glob('backup*'))[0]

/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_10_highV/alpha_yz_-10.0


In [31]:
fwp_spl = spl.Splopter(fwp_backup_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                       store_dataframe_fl=True, check_voltage_error_fl=False)

fwp_spl.prepare(find_se_temp_fl=False, homogenise_fl=True, backup_concat_fl=False)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_10_highV/alpha_yz_-10.0/backup_20201120-1751) doesn't seem to be valid.
Continuing anyway.


In [33]:
losw_plot_ds = lowsweep_ds.sel(theta_p=0.0, recession=0.0, theta=10.0)
pw_plot_ds = piecewise_ds.sel(theta=10.0)

fig, ax = plt.subplots()
splopter.iv_data.plot(ax=ax, label=r'floating $V_{wall}$')
fwp_spl.iv_data.plot(ax=ax, label=r'fixed $V_{wall}$')
(-pw_plot_ds['current']).plot.line(x='voltage', hue='theta', ax=ax, label='piecewise')
losw_plot_ds['current'].plot.line(x='voltage', ax=ax, label='low-V sweep')
ax.axhline(y=0, **c.AX_LINE_DEFAULTS)
ax.set_xlim(-26, 0)
ax.set_ylim(-50, 10)
ax.legend()

## Compare across angles!

In [34]:
fwp_spl_dirs = {
    8.0: spice_dir / all_run_dirs[scans[6]][1],
    10.0: spice_dir / all_run_dirs[scans[5]][0],
    12.0: spice_dir / all_run_dirs[scans[6]][0],
}

for k, v in fwp_spl_dirs.items():
    print(k, v)

8.0 /home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_8-12_highV/alpha_yz_-8.0
10.0 /home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_10_highV/alpha_yz_-10.0
12.0 /home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_8-12_highV/alpha_yz_-12.0


In [35]:
fwp_spls = {}
for theta, fwp_spl_dir in fwp_spl_dirs.items():
    bups = list(fwp_spl_dir.glob('backup*'))
    bups.sort()
    final_state_path = fwp_spl_dir / bups[-1].name
    
    fwp_spl = spl.Splopter(final_state_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                           store_dataframe_fl=True, check_voltage_error_fl=False)

    fwp_spl.prepare(find_se_temp_fl=False, homogenise_fl=True, backup_concat_fl=False)
    fwp_spls[theta] = fwp_spl

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_8-12_highV/alpha_yz_-8.0/backup_20201120-2311) doesn't seem to be valid.
Continuing anyway.
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_10_highV/alpha_yz_-10.0/backup_20201120-1751) doesn't seem to be valid.
Continuing anyway.
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_8-12_highV/alpha_yz_-12.0/backup_20201120-1629) doesn't seem to be valid.
Continuing anyway.


In [37]:
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True, figsize=[10,6])

for i, theta in enumerate(fwp_spls):
    ax = axes[i]
    pw_plot_ds = piecewise_ds.sel(theta=theta)
    
    fwp_spls[theta].iv_data.plot(ax=ax, label=r'fixed $V_{wall}$')
    (-pw_plot_ds['current']).plot.line(x='voltage', hue='theta', ax=ax, label='piecewise')
    
    ax.axhline(y=0, **c.AX_LINE_DEFAULTS)
    ax.set_title(r'$\theta = {}$'.format(theta))
    ax.set_xlim(-26, 0)
    ax.set_ylim(-50, 10)
    ax.legend()
    
fig.tight_layout()